# Задание 3: Сравнение производительности CPU vs CUDA

## 3.0. Импорт pytorch и создание девайса

In [1]:
import torch

# Введём устройство, на котором будут работать тензоры.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Тензоры на {device}.")

Тензоры на cuda:0.


## 3.1. Подготовка данных

In [4]:
matrix64 = torch.randint(1, 250, (64, 1024, 1024)).to(device)
matrix128 = torch.randint(1, 250, (128, 512, 512)).to(device)
matrix256 = torch.randint(1, 250, (256, 256, 256)).to(device)

## 3.2. Функция измерения времени

In [222]:
import time

def get_work_time(operation_type, tensor1, tensor2=None, func=None):
    """
    Измеряет время выполнения операции на CPU или GPU.
    """
    if tensor1.device == torch.device("cpu"):
        start_time = time.time()
        
        if operation_type == 'two_tensors':
            func(tensor1, tensor2)
        elif operation_type == 'permute':
            torch.permute(tensor1, (2, 1, 0))
        elif operation_type == 'sum':
            torch.sum(tensor1)

        end_time = time.time()
        elapsed_time = (end_time - start_time) * 1000 # Переводим в мс
    else:
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)
        
        start_event.record()
        
        if operation_type == 'two_tensors':
            func(tensor1, tensor2)
        elif operation_type == 'permute':
            torch.permute(tensor1, (2, 1, 0))
        elif operation_type == 'sum':
            torch.sum(tensor1)
            
        end_event.record()
        torch.cuda.synchronize()
        elapsed_time = start_event.elapsed_time(end_event)
    
    return round(elapsed_time, 1)

## 3.3. Сравнение операций

#### Матричное умножение

In [223]:
adapted_matrix256 = matrix256.reshape(256, -1).to(torch.float32)
adapted_matrix128 = matrix128.reshape(-1, 512).to(torch.float32)

mat_mul_gpu_time = get_work_time(
    'two_tensors',
    adapted_matrix256, 
    adapted_matrix128, 
    torch.matmul
)
mat_mul_cpu_time = get_work_time(
    'two_tensors',
    adapted_matrix256.to("cpu"),
    adapted_matrix128.to("cpu"),
    torch.matmul
)

mat_mul_diff = round(mat_mul_cpu_time / mat_mul_gpu_time, 2)

#### Поэлементное сложение

In [224]:
matrix64_cpu = matrix64.to(torch.device("cpu"))

add_gpu_time = get_work_time('two_tensors', matrix64, matrix64, torch.add)
add_cpu_time = get_work_time('two_tensors', matrix64_cpu, matrix64_cpu, torch.add)

add_diff = round(add_cpu_time / add_gpu_time, 1)

#### Поэлеметное умножение

In [238]:
mul_gpu_time = get_work_time('two_tensors', matrix64, matrix64, torch.mul)
mul_cpu_time = get_work_time('two_tensors', matrix64_cpu, matrix64_cpu, torch.mul)

mul_diff = round(mul_cpu_time / mul_gpu_time, 1)

#### Транспонирование

In [226]:
permute_gpu_time = get_work_time('permute', matrix64)
permute_cpu_time = get_work_time('permute', matrix64.to(torch.device("cpu")))

try:
    permute_diff = round(permute_cpu_time / permute_gpu_time, 1)
except ZeroDivisionError:
    permute_diff = 0

#### Вычисление суммы всех элементов

In [236]:
sum_gpu_time = get_work_time("sum", matrix64)
sum_cpu_time = get_work_time("sum", matrix64)

sum_diff = round(sum_cpu_time / sum_gpu_time, 1)

### Результат в табличном виде

In [240]:
print(f"""
Операция            | CPU (мс) | GPU (мс) | Ускорение
Матричное умножение | {mat_mul_cpu_time}  | {mat_mul_gpu_time} | {mat_mul_diff}
Сложение            | {add_cpu_time}  | {add_gpu_time} | {add_diff}
Умножение           | {mul_cpu_time} | {mul_gpu_time} | {mul_diff}
Транспонирование    | {permute_cpu_time}  | {permute_gpu_time} | {permute_diff}
Сумма элементов     | {sum_cpu_time}  | {sum_gpu_time} | {sum_diff}
""")


Операция            | CPU (мс) | GPU (мс) | Ускорение
Матричное умножение | 79.3  | 23.7 | 3.35
Сложение            | 44.6  | 4.9 | 9.1
Умножение           | 77.1 | 135.5 | 0.6
Транспонирование    | 0.0  | 0.0 | 0
Сумма элементов     | 59.8  | 58.9 | 1.0

